In [2]:
from text_preprocessor import CorpusToEmbedding

import pandas as pd

df = pd.read_csv("./preprocessed/train.csv")

import pickle

with open("preprocessed/text_object", 'rb') as f:
    text_obj = pickle.load(f)
    f.close()

In [3]:
df["text_tokens"] = text_obj.tokens
df["labels_tokens"] = text_obj.labels

In [4]:
df

,Unnamed: 0,id,img,label,text,labels,text_tokens,labels_tokens
0,0,42953,img/42953.png,0,its their character not their color that matters,"['person', 'person']","[19, 45, 730, 34, 45, 1549, 11, 619, 999999, 9...","[361, 361, 999999]"
1,1,23058,img/23058.png,0,don't be afraid to love again everyone is not ...,"['person', 'person', 'person', 'tie']","[98, 34, 38, 4840, 5, 650, 177, 584, 13, 34, 6...","[361, 361, 361]"
2,2,13894,img/13894.png,0,putting bows on your pet,"['cat', 'cat', 'bed']","[1629, 8139, 15, 54, 8544, 999999, 999999, 999...","[3511, 3511, 2818]"
3,3,37408,img/37408.png,0,i love everything and everybody! except for sq...,"['dog', 'dog']","[650, 749, 3, 3409, 1231, 14, 23171, 2992, 231...","[2369, 2369, 999999]"
4,4,82403,img/82403.png,0,"everybody loves chocolate chip cookies, even h...","['person', 'tie', 'tie']","[3409, 650, 7037, 7027, 435739, 134, 60393, 99...","[361, 4540, 4540]"
...,...,...,...,...,...,...,...,...
8495,8495,10423,img/10423.png,1,nobody wants to hang auschwitz me,"['person', 'tie', 'tie', 'bed', 'bed', 'bed']","[4014, 405, 5, 8974, 448544, 103, 999999, 9999...","[361, 4540, 4540]"
8496,8496,98203,img/98203.png,1,when god grants you a child after 20 years of ...,"['person', 'person', 'person', 'person', 'pers...","[71, 3730, 2590, 30, 815, 79, 163, 144, 4, 54,...","[361, 361, 361]"
8497,8497,36947,img/36947.png,1,gays on social media: equality! body positivit...,['person'],"[2635, 15, 687, 3394, 4683, 561, 41234, 650, 6...","[361, 999999, 999999]"
8498,8498,16492,img/16492.png,1,having a bad day? you could be a siamese twin ...,['person'],"[282, 816, 193, 30, 169, 38, 282629, 8287, 462...","[361, 999999, 999999]"


In [6]:
from torchvision import datasets, transforms
import torch

In [7]:
import numpy as np

df["new_labels"] = np.where(df.label == 0, 1, 0)
df["new_id"] = df.id.astype(str) # for lexsorting

In [9]:
df = df.sort_values(["new_labels", "new_id"])

In [10]:
texts = torch.utils.data.TensorDataset(torch.tensor(df.text_tokens), torch.tensor(df.new_labels))

In [26]:
labels = torch.utils.data.TensorDataset(torch.tensor(df.labels_tokens), torch.tensor(df.new_labels))

In [11]:
transformations = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

images = datasets.ImageFolder("./preprocessed/img", transform=transformations)

In [14]:
from torch.utils.data.sampler import RandomSampler

random_sampler = RandomSampler(images)

In [27]:
image_loader = torch.utils.data.DataLoader(
    images, batch_size=32, sampler=random_sampler
)
text_loader = torch.utils.data.DataLoader(texts, batch_size=32, sampler=random_sampler)

labels_loader = torch.utils.data.DataLoader(labels, batch_size=32, sampler=random_sampler)

In [24]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        drp = 0.1
        n_classes = 2

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)

        self.embedding = nn.Embedding.from_pretrained(torch.FloatTensor(text_obj.weights))
        self.embedding.requires_grad = False

        self.lstm = nn.LSTM(300, 30, bidirectional=False, batch_first=True)
        self.text_linear = nn.Sequential(
            nn.Linear(30, 15),
            nn.Linear(15, n_classes)
        )

        self.cnn_layer1 = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(drp),
            # Layer 2
            nn.Conv2d(3, 3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Dropout(drp),
            # # Layer 3
            # nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=0),
            # # Out: 62, 29
            # nn.BatchNorm2d(32),
            # nn.ReLU(inplace=True),
            # nn.MaxPool2d(kernel_size=3, stride=2, padding=0),
            # # Out: 30, 14
            # nn.Dropout(drp),
        )

        self.fc1 = nn.Linear(224 * 224 * 3, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, n_classes)

    def forward(self, x):
        # x = torch.unsqueeze(x, dim=1)
        x = x[1]
        h_embedding = self.embedding(x)
        h_lstm, _ = self.lstm(h_embedding)
        out = self.text_linear(h_lstm)
        # out1 = self.cnn_layer1(x)
        # out2 = self.cnn_layer2(x)
        # full_out = torch.cat([out1, out2], dim=1)
        # full_out = full_out.view(full_out.size(0), -1)
        # x = torch.flatten(out1, start_dim=1)
        # x = self.dropout(self.relu(self.fc1(x)))
        # x = self.dropout(self.relu(self.fc2(x)))
        # x = self.dropout(self.relu(self.fc3(x)))
        # x = self.dropout(self.relu(self.fc4(x)))
        # x = self.dropout(self.relu(self.fc5(x)))
        # out = self.out(x)
        return out

In [28]:
model = CNN()
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)

# loss_fn = nn.

for (i, j, k) in zip(image_loader, text_loader, labels_loader):
    image = i[0].to(device)
    text = j[0].to(device)
    output = model([image, text]) # hacky way to do this?
    print(output)
    break

tensor([[[-0.0141,  0.0636],
         [-0.0058,  0.0626],
         [-0.0065,  0.0760],
         ...,
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833]],

        [[-0.0206,  0.0794],
         [-0.0024,  0.0777],
         [ 0.0111,  0.0729],
         ...,
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833]],

        [[-0.0220,  0.0688],
         [-0.0138,  0.0707],
         [-0.0052,  0.0754],
         ...,
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833]],

        ...,

        [[-0.0032,  0.0778],
         [-0.0018,  0.0889],
         [ 0.0059,  0.0920],
         ...,
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833]],

        [[-0.0023,  0.0701],
         [ 0.0092,  0.0741],
         [ 0.0007,  0.0629],
         ...,
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833],
         [ 0.0066,  0.0833]],

        [[-0.0006,  0.0765],
       